In [1]:
import os
import re


info_files = [
    file_ for file_ in os.listdir(
        '~/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/input_vector'
    ) if file_.endswith('.info')
]

commands = set()
for info_file in info_files:
    # print(info_file)
    matches = re.findall(
        r'\[.*sys\.argv.*\]\s*\[(.*)\]',
        open(info_file, 'r').read()
    )
    # print(matches)

    if matches:
        text = matches[0]
        regex = r'\'(.*?)\''
        command = re.findall(regex, text)
        # print(' '.join(command))
        commands.add(' '.join(command[1:]))
        # print(command)
    else:
        # print('No command found')
        pass
    # print()

# print(len(info_files))
# print(len(commands))

for cmd in commands:
    print(cmd, '\n')
# print(commands)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '~/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/input_vector'

In [2]:
# generate sbatch script and launch the jobs
# add the smoothed values

In [3]:
import locale
import subprocess


def run_command(command):
    """ Run the command and directly output to the console. """
    process = subprocess.Popen(command, stdout=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.decode(locale.getdefaultlocale()[1]), end='')
    return process.poll()


if __name__ == '__main__':
    recipe_command = [
        'echo',
        'hello'
    ]

    run_command(recipe_command)


hello


In [4]:
import os
# sbash_template = os.path.realpath(
#     os.path.join(os.getcwd(), '..', 'jobs_scripts', 'template.sh')
# )
sbash_template = '~/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/jobs_scripts/template.sh'
sbash_template = open(sbash_template).read()

gpu_template_ = sbash_template.format(
    'auto_gpu',
    'gpu',
    '1',
    '16',
    '60G',
    '24:00:00',
    'id@umons.ac.be',
    'ALL',
    '#SBATCH --gres="gpu:2"',
    '{0}',
)

#SBATCH --job-name=all_ohv
#SBATCH --partition=days
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=16
#SBATCH --mem=60G
#SBATCH -t 4:00:00
#SBATCH --mail-user=id@umons.ac.be
#SBATCH --mail-type=ALL
hpc_template_ = sbash_template.format(
    'auto_hpc',
    'days',
    '1',
    '16',
    '60G',
    '24:00:00',
    'id@umons.ac.be',
    'ALL',
    '',
    '{0}',
)

def hpc_template(command):
    return hpc_template_.format(command)

def gpu_template(command):
    return gpu_template_.format(command)

def run_gpu(command, filename):
    open(filename, 'w').write(gpu_template(command))
    run_command('sbatch {0}'.format(filename).split())

# def metrics_commands(model, prefix):
#     return 'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} -sw 12 -sr 0.666666666 -df tuh ar gsz -tf tuh ar gsz --xlsx_file {1}_AR_GSZ_SMOOTHING_8_12.xlsx\n'.format(model, prefix)

def metrics_commands(model, prefix):
    # return 'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} -sw 12 -sr 0.666666666 -df tuh ar gsz -tf tuh ar gsz --xlsx_file {1}_AR_GSZ_SMOOTHING_8_12.xlsx\n'.format(model, prefix)
    return (
        # AR GSZ
        'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} '
        '-sw 12 -sr 0.666666666 -df tuh ar gsz -tf tuh ar gsz '
        '--xlsx_file {1}_AR_GSZ_SMOOTHING_8_12.xlsx\n'
        'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} '
        '-sw 21 -sr 0.7143 -df tuh ar gsz -tf tuh ar gsz '
        '--xlsx_file {1}_AR_GSZ_SMOOTHING_15_21.xlsx\n'
        'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} '
        '-sw 0 -df tuh ar gsz -tf tuh ar gsz '
        '--xlsx_file {1}_AR_GSZ_NO_SMOOTHING.xlsx\n'
        # AR
        'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} '
        '-sw 12 -sr 0.666666666 -df tuh ar -tf tuh ar '
        '--xlsx_file {1}_AR_SMOOTHING_8_12.xlsx\n'
        'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} '
        '-sw 21 -sr 0.7143 -df tuh ar -tf tuh ar '
        '--xlsx_file {1}_AR_SMOOTHING_15_21.xlsx\n'
        'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} '
        '-sw 0 -df tuh ar -tf tuh ar '
        '--xlsx_file {1}_AR_NO_SMOOTHING.xlsx\n'
        # AR and LE
        'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} '
        '-sw 12 -sr 0.666666666 -df tuh ar_le -tf tuh ar_le '
        '--xlsx_file {1}_AR_LE_SMOOTHING_8_12.xlsx\n'
        'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} '
        '-sw 21 -sr 0.7143 -df tuh ar_le -tf tuh ar_le '
        '--xlsx_file {1}_AR_LE_SMOOTHING_15_21.xlsx\n'
        'srun -N1 -n1 -c1 --exclusive python3 src/metrics.py {0} '
        '-sw 0 -df tuh ar_le -tf tuh ar_le '
        '--xlsx_file {1}_AR_LE_NO_SMOOTHING.xlsx\n'
    ).format(model, prefix)

print(metrics_commands('model', 'reports/metrics/model_1'))

# run_gpu('echo hello', 'test')

srun -N1 -n1 -c1 --exclusive python3 src/metrics.py model -sw 12 -sr 0.666666666 -df tuh ar gsz -tf tuh ar gsz --xlsx_file reports/metrics/model_1_AR_GSZ_SMOOTHING_8_12.xlsx
srun -N1 -n1 -c1 --exclusive python3 src/metrics.py model -sw 21 -sr 0.7143 -df tuh ar gsz -tf tuh ar gsz --xlsx_file reports/metrics/model_1_AR_GSZ_SMOOTHING_15_21.xlsx
srun -N1 -n1 -c1 --exclusive python3 src/metrics.py model -sw 0 -df tuh ar gsz -tf tuh ar gsz --xlsx_file reports/metrics/model_1_AR_GSZ_NO_SMOOTHING.xlsx
srun -N1 -n1 -c1 --exclusive python3 src/metrics.py model -sw 12 -sr 0.666666666 -df tuh ar -tf tuh ar --xlsx_file reports/metrics/model_1_AR_SMOOTHING_8_12.xlsx
srun -N1 -n1 -c1 --exclusive python3 src/metrics.py model -sw 21 -sr 0.7143 -df tuh ar -tf tuh ar --xlsx_file reports/metrics/model_1_AR_SMOOTHING_15_21.xlsx
srun -N1 -n1 -c1 --exclusive python3 src/metrics.py model -sw 0 -df tuh ar -tf tuh ar --xlsx_file reports/metrics/model_1_AR_NO_SMOOTHING.xlsx
srun -N1 -n1 -c1 --exclusive python3 s

In [5]:
os.chdir('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/')
print(os.getcwd())
# Model 1
model_1 ='srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar gsz -tf tuh ar gsz --files_prefix model_1_inputs -th 0.8 -rp -a -y\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_1_inputs.info --split_train  --files_prefix model_1_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_1_model.info', 'reports/metrics/model_1')

# Model 10
model_10 ='srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar_le -tf tuh ar_le --files_prefix model_10_inputs -th 0.8 -rp -a -y\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_10_inputs.info --split_train  --files_prefix model_10_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_10_model.info', 'reports/metrics/model_10')

# Model 11
model_11 ='srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar -tf tuh ar --files_prefix model_11_inputs -th 0.8 -rp -a -y\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_11_inputs.info --split_train  --files_prefix model_11_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_11_model.info', 'reports/metrics/model_11')

# Model 2
model_2 = 'srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar gsz -tf tuh ar gsz --files_prefix model_2_inputs -f line_length min -th 0.8 -rp -a -y\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_2_inputs.info --split_train --files_prefix model_2_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_2_model.info', 'reports/metrics/model_2')

# Model 3
model_3 = 'srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar gsz -tf tuh ar gsz --files_prefix model_3_inputs -th 0.8 -rp -a -y -f MIN MEAN MAX VARIANCE KURTOSIS SKEWNESS INTER_QUARTILE_RANGE HIGUCHI_FRACTAL_DIMENSION HJORTH_ACTIVITY HJORTH_MOBILITY HJORTH_COMPLEXITY PETROSIAN_FRACTAL_DIMENSION PAUL_FRACTAL_DIMENSION LINE_LENGTH MULTITAPER_DELTA_05HZ_4HZ_POWER MULTITAPER_THETA_4HZ_8HZ_POWER MULTITAPER_ALPHA_8HZ_12HZ_POWER MULTITAPER_BETA_12HZ_30HZ_POWER MULTITAPER_GAMMA_30HZ_100HZ_POWER MULTITAPER_EPILEPSY_2HZ_4HZ_POWER MULTITAPER_EPILEPSY_1HZ_5HZ_POWER MULTITAPER_EPILEPSY_0HZ_6HZ_POWER MULTITAPER_TOTAL_POWER MULTITAPER_DELTA_05HZ_4HZ_POWER_RATIO MULTITAPER_THETA_4HZ_8HZ_POWER_RATIO MULTITAPER_ALPHA_8HZ_12HZ_POWER_RATIO MULTITAPER_BETA_12HZ_30HZ_POWER_RATIO MULTITAPER_GAMMA_30HZ_100HZ_POWER_RATIO MULTITAPER_EPILEPSY_2HZ_4HZ_POWER_RATIO MULTITAPER_EPILEPSY_1HZ_5HZ_POWER_RATIO MULTITAPER_EPILEPSY_0HZ_6HZ_POWER_RATIO SPECTRAL_CENTROID SPECTRAL_FLATNESS\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_3_inputs.info --split_train --files_prefix model_3_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_3_model.info', 'reports/metrics/model_3')

# Model 4
model_4 = 'srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar gsz -tf tuh ar gsz --files_prefix model_4_inputs -th 0.8 -rp -a -y -f MIN MEAN MAX VARIANCE KURTOSIS SKEWNESS INTER_QUARTILE_RANGE HIGUCHI_FRACTAL_DIMENSION HJORTH_ACTIVITY HJORTH_MOBILITY HJORTH_COMPLEXITY PETROSIAN_FRACTAL_DIMENSION PAUL_FRACTAL_DIMENSION LINE_LENGTH MULTITAPER_DELTA_05HZ_4HZ_POWER MULTITAPER_THETA_4HZ_8HZ_POWER MULTITAPER_ALPHA_8HZ_12HZ_POWER MULTITAPER_BETA_12HZ_30HZ_POWER MULTITAPER_GAMMA_30HZ_100HZ_POWER MULTITAPER_EPILEPSY_2HZ_4HZ_POWER MULTITAPER_EPILEPSY_1HZ_5HZ_POWER MULTITAPER_EPILEPSY_0HZ_6HZ_POWER MULTITAPER_TOTAL_POWER MULTITAPER_DELTA_05HZ_4HZ_POWER_RATIO MULTITAPER_THETA_4HZ_8HZ_POWER_RATIO MULTITAPER_ALPHA_8HZ_12HZ_POWER_RATIO MULTITAPER_BETA_12HZ_30HZ_POWER_RATIO MULTITAPER_GAMMA_30HZ_100HZ_POWER_RATIO MULTITAPER_EPILEPSY_2HZ_4HZ_POWER_RATIO MULTITAPER_EPILEPSY_1HZ_5HZ_POWER_RATIO MULTITAPER_EPILEPSY_0HZ_6HZ_POWER_RATIO SPECTRAL_CENTROID SPECTRAL_FLATNESS -m F7-T3\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_4_inputs.info --split_train --files_prefix model_4_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_4_model.info', 'reports/metrics/model_4')

# Model 5
model_5 = 'srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar gsz -tf tuh ar gsz --files_prefix model_5_inputs -th 0.8 -rp  -a -y -f MIN MEAN MAX VARIANCE KURTOSIS SKEWNESS INTER_QUARTILE_RANGE HIGUCHI_FRACTAL_DIMENSION HJORTH_ACTIVITY HJORTH_MOBILITY HJORTH_COMPLEXITY PETROSIAN_FRACTAL_DIMENSION PAUL_FRACTAL_DIMENSION LINE_LENGTH MULTITAPER_DELTA_05HZ_4HZ_POWER MULTITAPER_THETA_4HZ_8HZ_POWER MULTITAPER_ALPHA_8HZ_12HZ_POWER MULTITAPER_BETA_12HZ_30HZ_POWER MULTITAPER_GAMMA_30HZ_100HZ_POWER MULTITAPER_EPILEPSY_2HZ_4HZ_POWER MULTITAPER_EPILEPSY_1HZ_5HZ_POWER MULTITAPER_EPILEPSY_0HZ_6HZ_POWER MULTITAPER_TOTAL_POWER MULTITAPER_DELTA_05HZ_4HZ_POWER_RATIO MULTITAPER_THETA_4HZ_8HZ_POWER_RATIO MULTITAPER_ALPHA_8HZ_12HZ_POWER_RATIO MULTITAPER_BETA_12HZ_30HZ_POWER_RATIO MULTITAPER_GAMMA_30HZ_100HZ_POWER_RATIO MULTITAPER_EPILEPSY_2HZ_4HZ_POWER_RATIO MULTITAPER_EPILEPSY_1HZ_5HZ_POWER_RATIO MULTITAPER_EPILEPSY_0HZ_6HZ_POWER_RATIO SPECTRAL_CENTROID SPECTRAL_FLATNESS -m T3-C3\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_5_inputs.info --split_train --files_prefix model_5_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_5_model.info', 'reports/metrics/model_5')

# Model 6
model_6 = 'srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar gsz -tf tuh ar gsz --files_prefix model_6_inputs -th 0.8 -rp -a -y -f MIN MEAN MAX VARIANCE KURTOSIS SKEWNESS INTER_QUARTILE_RANGE HIGUCHI_FRACTAL_DIMENSION HJORTH_ACTIVITY HJORTH_MOBILITY HJORTH_COMPLEXITY PETROSIAN_FRACTAL_DIMENSION PAUL_FRACTAL_DIMENSION LINE_LENGTH MULTITAPER_DELTA_05HZ_4HZ_POWER MULTITAPER_THETA_4HZ_8HZ_POWER MULTITAPER_ALPHA_8HZ_12HZ_POWER MULTITAPER_BETA_12HZ_30HZ_POWER MULTITAPER_GAMMA_30HZ_100HZ_POWER MULTITAPER_EPILEPSY_2HZ_4HZ_POWER MULTITAPER_EPILEPSY_1HZ_5HZ_POWER MULTITAPER_EPILEPSY_0HZ_6HZ_POWER MULTITAPER_TOTAL_POWER MULTITAPER_DELTA_05HZ_4HZ_POWER_RATIO MULTITAPER_THETA_4HZ_8HZ_POWER_RATIO MULTITAPER_ALPHA_8HZ_12HZ_POWER_RATIO MULTITAPER_BETA_12HZ_30HZ_POWER_RATIO MULTITAPER_GAMMA_30HZ_100HZ_POWER_RATIO MULTITAPER_EPILEPSY_2HZ_4HZ_POWER_RATIO MULTITAPER_EPILEPSY_1HZ_5HZ_POWER_RATIO MULTITAPER_EPILEPSY_0HZ_6HZ_POWER_RATIO SPECTRAL_CENTROID SPECTRAL_FLATNESS -m T3-T5\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_6_inputs.info --split_train --files_prefix model_6_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_6_model.info', 'reports/metrics/model_6')

# Model 7
model_7 = 'srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar_le -tf tuh ar_le --files_prefix model_7_inputs -th 0.8 -rp -a -y -f MIN MEAN MAX VARIANCE KURTOSIS SKEWNESS INTER_QUARTILE_RANGE HIGUCHI_FRACTAL_DIMENSION HJORTH_ACTIVITY HJORTH_MOBILITY HJORTH_COMPLEXITY PETROSIAN_FRACTAL_DIMENSION PAUL_FRACTAL_DIMENSION LINE_LENGTH MULTITAPER_DELTA_05HZ_4HZ_POWER MULTITAPER_THETA_4HZ_8HZ_POWER MULTITAPER_ALPHA_8HZ_12HZ_POWER MULTITAPER_BETA_12HZ_30HZ_POWER MULTITAPER_GAMMA_30HZ_100HZ_POWER MULTITAPER_EPILEPSY_2HZ_4HZ_POWER MULTITAPER_EPILEPSY_1HZ_5HZ_POWER MULTITAPER_EPILEPSY_0HZ_6HZ_POWER MULTITAPER_TOTAL_POWER MULTITAPER_DELTA_05HZ_4HZ_POWER_RATIO MULTITAPER_THETA_4HZ_8HZ_POWER_RATIO MULTITAPER_ALPHA_8HZ_12HZ_POWER_RATIO MULTITAPER_BETA_12HZ_30HZ_POWER_RATIO MULTITAPER_GAMMA_30HZ_100HZ_POWER_RATIO MULTITAPER_EPILEPSY_2HZ_4HZ_POWER_RATIO MULTITAPER_EPILEPSY_1HZ_5HZ_POWER_RATIO MULTITAPER_EPILEPSY_0HZ_6HZ_POWER_RATIO SPECTRAL_CENTROID SPECTRAL_FLATNESS -m F7-T3\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_7_inputs.info --split_train --files_prefix model_7_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_7_model.info', 'reports/metrics/model_7')

# Model 8
model_8 = 'srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar_le -tf tuh ar_le --files_prefix model_8_inputs -th 0.8 -rp  -a -y -f MIN MEAN MAX VARIANCE KURTOSIS SKEWNESS INTER_QUARTILE_RANGE HIGUCHI_FRACTAL_DIMENSION HJORTH_ACTIVITY HJORTH_MOBILITY HJORTH_COMPLEXITY PETROSIAN_FRACTAL_DIMENSION PAUL_FRACTAL_DIMENSION LINE_LENGTH MULTITAPER_DELTA_05HZ_4HZ_POWER MULTITAPER_THETA_4HZ_8HZ_POWER MULTITAPER_ALPHA_8HZ_12HZ_POWER MULTITAPER_BETA_12HZ_30HZ_POWER MULTITAPER_GAMMA_30HZ_100HZ_POWER MULTITAPER_EPILEPSY_2HZ_4HZ_POWER MULTITAPER_EPILEPSY_1HZ_5HZ_POWER MULTITAPER_EPILEPSY_0HZ_6HZ_POWER MULTITAPER_TOTAL_POWER MULTITAPER_DELTA_05HZ_4HZ_POWER_RATIO MULTITAPER_THETA_4HZ_8HZ_POWER_RATIO MULTITAPER_ALPHA_8HZ_12HZ_POWER_RATIO MULTITAPER_BETA_12HZ_30HZ_POWER_RATIO MULTITAPER_GAMMA_30HZ_100HZ_POWER_RATIO MULTITAPER_EPILEPSY_2HZ_4HZ_POWER_RATIO MULTITAPER_EPILEPSY_1HZ_5HZ_POWER_RATIO MULTITAPER_EPILEPSY_0HZ_6HZ_POWER_RATIO SPECTRAL_CENTROID SPECTRAL_FLATNESS -m T3-C3\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_8_inputs.info --split_train --files_prefix model_8_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_8_model.info', 'reports/metrics/model_8')

# Model 9
model_9 = 'srun python3 src/save_input_vectors.py /home_nfs/stragierv/dataset.h5 -df tuh ar_le -tf tuh ar_le --files_prefix model_9_inputs -th 0.8 -rp -a -y -f MIN MEAN MAX VARIANCE KURTOSIS SKEWNESS INTER_QUARTILE_RANGE HIGUCHI_FRACTAL_DIMENSION HJORTH_ACTIVITY HJORTH_MOBILITY HJORTH_COMPLEXITY PETROSIAN_FRACTAL_DIMENSION PAUL_FRACTAL_DIMENSION LINE_LENGTH MULTITAPER_DELTA_05HZ_4HZ_POWER MULTITAPER_THETA_4HZ_8HZ_POWER MULTITAPER_ALPHA_8HZ_12HZ_POWER MULTITAPER_BETA_12HZ_30HZ_POWER MULTITAPER_GAMMA_30HZ_100HZ_POWER MULTITAPER_EPILEPSY_2HZ_4HZ_POWER MULTITAPER_EPILEPSY_1HZ_5HZ_POWER MULTITAPER_EPILEPSY_0HZ_6HZ_POWER MULTITAPER_TOTAL_POWER MULTITAPER_DELTA_05HZ_4HZ_POWER_RATIO MULTITAPER_THETA_4HZ_8HZ_POWER_RATIO MULTITAPER_ALPHA_8HZ_12HZ_POWER_RATIO MULTITAPER_BETA_12HZ_30HZ_POWER_RATIO MULTITAPER_GAMMA_30HZ_100HZ_POWER_RATIO MULTITAPER_EPILEPSY_2HZ_4HZ_POWER_RATIO MULTITAPER_EPILEPSY_1HZ_5HZ_POWER_RATIO MULTITAPER_EPILEPSY_0HZ_6HZ_POWER_RATIO SPECTRAL_CENTROID SPECTRAL_FLATNESS -m T3-T5\nsrun --partition=gpu -N 1 -c 16 --mem=60G --gres="gpu:2" python3 src/train_xgb.py input_vectors/model_9_inputs.info --split_train --files_prefix model_9_model -y\n' + metrics_commands('/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection/models/model_9_model.info', 'reports/metrics/model_9')

models = [
    # ('model_1', model_1),
    ('model_2', model_2),
    # ('model_3', model_3),
    # ('model_4', model_4),
    # ('model_5', model_5),
    # ('model_6', model_6),
]

all_models = '\n'.join(
    [
        model_1,
        model_2,
        model_3,
        model_4,
        model_5,
        model_6,
        model_7,
        model_8,
        model_9,
    ],
)

# print(all_models)

# for model in models:
#     print(model[0] + ':\n', model[1], sep='', end='\n\n')
#     run_gpu(model[1], model[0])

# run_gpu(all_models, 'all_models_2021_Q1.sh')


/home_nfs/stragierv/TUH_SZ_v1.5.2/Epilepsy_Seizure_Detection


In [6]:
all_models = '\n'.join(
    [
        model_1,
        model_2,
        model_3,
        model_4,
        model_5,
        model_6,
        model_7,
        model_8,
        model_9,
        model_10,
        model_11,
    ],
)

# print(all_models)

# for model in models:
#     print(model[0] + ':\n', model[1], sep='', end='\n\n')
#     run_gpu(model[1], model[0])

# run_gpu(all_models, 'all_models_2021_Q1_(1-6).sh')
run_gpu(all_models, 'all_models_2021_Q1_(1-11).sh')
# run_gpu(all_models, 'all_models_2021_Q1_(7-10).sh')
# run_gpu(model_10, 'all_models_2021_Q1_(10).sh')


Submitted batch job 7341
